## InferSent

In [8]:
from sklearn.metrics.pairwise import cosine_similarity
from models import InferSent
import torch
import json,os,gc
import pickle

import pandas as pd
import numpy as np

import joblib

# Customized
path = 'C:\\Users\Ilias\\OneDrive\\Cours\\Semestre 07\\Linguistica\\practica_3\\'
#path = '/home/averkepasa/IL/'

In [2]:
class InferSent_corpus:
    def __init__(self,path):
            
            self.infersent = InferSent({'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': 2}) #because we use infersent2 pre-trained model
            self.infersent.load_state_dict(torch.load(path+'infersent.pkl'))
            self.infersent.set_w2v_path(path+'crawl-300d-2M.vec')
            self.embeddings = joblib.load(path+'embeddings.joblib')
            self.infersent.word_vec = joblib.load(path+'word_vec.joblib')

            self.sentences = open(path+"corpus.txt", "r",encoding='utf-8').readlines()
            with open(path+'documents.pickle', 'rb') as f:
                self.df = pickle.load(f)
            with open(path+'document_sents.pickle', 'rb') as f:
                self.df_sent = pickle.load(f)
                
    def find_similar(self,sent,n,time):
        embedded_query = self.infersent.encode([sent], tokenize=True) #query needs to be in an array
        similarity_matrix = cosine_similarity(embedded_query, self.embeddings)
        np.fill_diagonal(similarity_matrix, 0)
        similarities = similarity_matrix[0]
        if n == 1:
            return [np.argmax(similarities)]
        elif n is not None:
            return np.flip(similarities.argsort()[(-n*time):len(similarities)-(n*(time-1))][::1])

    def article_inf_from_sent(self,art_no):
        #sent=str(sent)
        #if sent[-1:]=='\n':
        #    sent = sent[:-1] #[:-1] we remove \n char due to encoding by lines
        
        article = self.df[self.df['article_no']==art_no]
        authors = article['authors']
        text = article['body']
        abstract = article['abstract']
        title = article['titles']
        return authors,text,abstract,title
    
    def proposal(self,sent,n,time=1):
        
        print(('\n Results of page '+str(time) + ' : \n'))
        for i in self.find_similar(sent,n,time):
            art_no = np.array(self.df_sent[self.df_sent['sentence'] == str(self.sentences[i])[:-1]]['article_id'])[0] #np.array to get the element as an itneger and not as a list
            print(('Article '+str(art_no)))
            print(self.sentences[i])
        value = input("To get more info on an article, enter its number. To see more results, enter 'More'. To make another search, enter 'Menu'. To go to previous page, enter 'Previous'.\n\n").lower()
        if value=='more':
            self.proposal(sent,n,time+1)
            return()
        if value=='menu':
#            menu()
            return()
        if value=='previous' or value=='p':
            if(time>1):
                self.proposal(sent,n,time-1)
                return()
            else:
                print('\nYou where to the first page. You will be sent to the menu.')
#                menu()
                return()             

        try:
            self.print_infos(int(value))
        except:
            print('Error. You need to enter a valid option or number.\nYou were sent back to the menu.')
 #           menu()
        return()
    def print_infos(self,art_no):
        authors,text,abstract,title = self.article_inf_from_sent(art_no)
        print('\nArticle n°' + str(art_no) +'.\n' + 'Authors : ' + np.array(authors)[0] + '.\n' + 'Title : ' + np.array(title)[0] + '\n\n' + 'Abstract : ' + np.array(abstract)[0])
        value = input("\nTo read the article's body, enter 'read'. To make another search, enter 'menu'.\n\n").lower()
        if value=='read' or value=='r':
            print(np.array(text)[0] + '\n\n')
  #          menu()
            return()
        elif value != 'menu':
            print('Youre choice where not valid. You were sent back to the menu')
    #        menu()
        return()
        

In [20]:
class Corpus_questions:

    def __init__(self,infersent_corpus):
        self.infersent_corpus=infersent_corpus
        self.embeddings_questions = joblib.load('embeddings_questions.joblib')
        self.df_questions = joblib.load('df_questions.joblib')
        
    def process_query(self,query,t):
        value = 'tmp'
        if t==1:
            ind = self.find_similar_question(query,1)
            value = input('\nThis question is not recognized by the database.\n Did you mean : ' + np.array(self.df_questions['questions'].loc[ind])[0] + '(yes/no)\n').lower()
            if value=='yes' or value=='y':
                print('Answer : ' + np.array(self.df_questions['answers'].loc[ind])[0])
      #          menu()
                return()
        if value=='no' or value=='No' or value=='n' or value=='N' or value=='tmp':
            value = input('You could find the desired by exploring our search engine. Would you like to do it ? (yes/no)\n').lower()
            if value=='no':
                print('You were sent to the menu\n\n')
     #               menu()
                return()
            if value=='yes' or value=='y':
                self.infersent_corpus.proposal(query,5)
                return()
            print('Invalid choice. You were sent to the menu\n\n')
    #            menu()
            return()
    

    def find_similar_question(self,sent,n):
            embedded_query = self.infersent_corpus.infersent.encode([sent], tokenize=True) #query needs to be in an array
            similarity_matrix = cosine_similarity(embedded_query, self.embeddings_questions)
            np.fill_diagonal(similarity_matrix, 0)
            similarities = similarity_matrix[0]
            if n == 1:
                return [np.argmax(similarities)]
            elif n is not None:
                return np.flip(similarities.argsort()[-n:][::1])

    def add_question_answer(self):
        #We won't check if the question is already entered. Actually, the more forms of the same question we have, the better it would be for InferSent.
        value = input('\nEnter your question.\n') 
        value2 = input('\nPlease enter your answer.\n')
        self.df_questions.loc[len(self.df_questions)] = list([value,value2])
        embedded_query = self.infersent_corpus.infersent.encode([value], tokenize=True)
        self.embeddings_questions = np.append(self.embeddings_questions,embedded_query,axis=0)
        #print(self.embeddings_questions)
        joblib.dump(self.embeddings_questions,'embeddings_questions.joblib', compress=True) #Pickle can't read this variable so we use joblib (maybe too big...?)
        joblib.dump(self.df_questions,'df_questions.joblib', compress=True) #Pickle can't read this variable so we use joblib (maybe too big...?)


        value = input('\n Thanks you very much. Would you like to add another question ? (yes/no)\n').lower()
        if value=='y' or value=='yes':
            self.add_question_answer()
            return()
        else:
            print('\nYou were sent back to the menu.\n')
            #menu()
            return()


## Table class

In [13]:
# Big dictionary, each entry is a dimension with a tuple of values:
# (value_list, description)
# -value_list is a list of values [38, 39.5, 1] or ["Parks", "Countryside"] (no repetitions)
# -description = description of the meaning of the dimension

class Table:
    def __init__(self):
        # Dictionary
        self.table = {}

    ### Add values ###

    # Add new value to a dimension
    def add(self, dimension, value, description = None):
        dimension = dimension.lower()
        value = value.lower()
        if dimension in self.table.keys():
            value_list, _ = self.table[dimension]
            if value not in value_list:
                value_list.append(value)
            if description is not None:
                self.table[dimension] = (value_list, description)
        else:
            self.table[dimension] = ([value], description)
    
    # add value without description
    def addValue(self, dimension, value):
        dimension = dimension.lower()
        value = value.lower()
        if dimension in self.table.keys():
            value_list, _ = self.table[dimension]
            if value not in value_list:
                value_list.append(value)
        else:
            self.table[dimension] = ([value], "Default dimension")
            
    ### Delete values ###

    # delete a dimension
    def deleteDimension(self, dimension):
        dimension = dimension.lower()
        if dimension in self.table.keys():
            del self.table[dimension]
            
    # delete a value from a dimension
    def deleteValue(self, dimension, value):
        dimension = dimension.lower()
        value = value.lower()
        if dimension in self.table.keys():
            value_list, _ = self.table[dimension]
            value_list.remove(value)
            
    ### Update values ###

    # update description from a dimension
    def updateDescription(self, dimension, description):
        dimension = dimension.lower()
        if dimension in self.table.keys():
            (value_list, _) = self.table[dimension]
            self.table[dimension] = (value_list, description)
        else:
            self.table[dimension] = ([], description)
            
    ### Get values ###

    # get all dimensions (rows)
    def getDimensions(self):
        return list(self.table.keys())

    # given the dimension returns all the values
    def getValues(self, dimension):
        dimension = dimension.lower()
        if dimension in self.table.keys():
            value_list, _ = self.table[dimension]
            return value_list
        else:
            print("getValues-Error: dimension ", dimension, " not found")
            return []

    # given the dimension returns the description
    def getDescription(self, dimension):
        dimension = dimension.lower()
        if dimension in self.table.keys():
            _, description = self.table[dimension]
            return description
        else:
            print("getDescription-Error: dimension ", dimension, " not found")
            return ""

    ### Functionality ###

    def processQuery(self, query):
        keywords = query.lower().split()

        # Parse bad temperatures
        if ("temperature" in keywords and "bad" in keywords) or ("fever" in keywords):
            if "all" in keywords:
                return self.getValues("Bad temperature")
            # It returns the minimum of the bad temperature
            temperatures = [float(i) for i in self.getValues("Bad temperature")]
            return str(min(temperatures))
        
        # Parse good places
        elif ("good" in keywords and "places" in keywords) or ("good" in keywords and "go" in keywords):
            if "all" in keywords:
                return self.getValues("Good places")
            # It returns the minimum of the bad temperature
            elif len([kw for kw in keywords if kw in self.getValues("Good places")]) > 0:
                return "Yes"
            elif len(self.getValues("Good places")) > 0:
                return self.getValues("Good places")[0]

        # Parse bad places
        elif ("bad" in keywords and "places" in keywords) or ("bad" in keywords and "go" in keywords):
            if "all" in keywords:
                return self.getValues("Bad places")
            # It returns the minimum of the bad temperature
            elif len([kw for kw in keywords if kw in self.getValues("Bad places")]) > 0:
                return "Yes"
            elif len(self.getValues("Bad places")) > 0:
                return self.getValues("Bad places")[0]

        # Parse covid description
        elif ("what is" in keywords and ("covid" in keywords or "covid-19" in keywords or "coronavirus" in keywords)):
            if "all" in keywords:
                return self.getValues("covid description")
            # It returns the minimum of the bad temperature
            elif len([kw for kw in keywords if kw in self.getValues("covid description")]) > 0:
                return "Yes"
            elif len(self.getValues("covid description")) > 0:
                return self.getValues("covid description")[0]

        # Parse who infects
        elif ("who" in keywords):
            if "all" in keywords:
                return self.getValues("who infects by covid")
            # It returns the minimum of the bad temperature
            elif len([kw for kw in keywords if kw in self.getValues("who infects by covid")]) > 0:
                return "Yes"
            elif len(self.getValues("who infects by covid")) > 0:
                return self.getValues("who infects by covid")[0]

        # Parse who infects
        elif ("who" in keywords):
            if "all" in keywords:
                return self.getValues("who infects by covid")
            # It returns the minimum of the bad temperature
            elif len([kw for kw in keywords if kw in self.getValues("who infects by covid")]) > 0:
                return "Yes"
            elif len(self.getValues("who infects by covid")) > 0:
                return self.getValues("who infects by covid")[0]

        # Parse risk people
        elif ("risk" in keywords and ("who" in keywords or "people" in keywords)):
            if "all" in keywords:
                return self.getValues("risk people")
            # It returns the minimum of the bad temperature
            elif len([kw for kw in keywords if kw in self.getValues("risk people")]) > 0:
                return "Yes"
            elif len(self.getValues("risk people")) > 0:
                return self.getValues("risk people")[0]

        # Parse ventilation tips
        elif ("ventilation" in keywords and ("tip" in keywords or "tips" in keywords or "advice" in keywords)):
            if "all" in keywords:
                return self.getValues("Ventilation tips")
            # It returns the minimum of the bad temperature
            elif len([kw for kw in keywords if kw in self.getValues("Ventilation tips")]) > 0:
                return "Yes"
            elif len(self.getValues("Ventilation tips")) > 0:
                return self.getValues("Ventilation tips")[0]

        # Parse prevention advice
        elif ("tips" in keywords or "tip" in keywords or "advice" in keywords or "prevention" in keywords or "avoid covid" in keywords):
            if "all" in keywords:
                return self.getValues("prevention advice")
            # It returns the minimum of the bad temperature
            elif len([kw for kw in keywords if kw in self.getValues("prevention advice")]) > 0:
                return "Yes"
            elif len(self.getValues("prevention advice")) > 0:
                return self.getValues("prevention advice")[0]

        # Parse how spreads
        elif ("how" in keywords and ("spread" in keywords or "spreads" in keywords or "propagate" in keywords or "propagates" in keywords)):
            if "all" in keywords:
                return self.getValues("how spreads")
            # It returns the minimum of the bad temperature
            elif len([kw for kw in keywords if kw in self.getValues("how spreads")]) > 0:
                return "Yes"
            elif len(self.getValues("how spreads")) > 0:
                return self.getValues("how spreads")[0]

        # Parse when symptoms appear
        elif (("when" in keywords and "symptoms" in keywords) or ("appear" in keywords and "symptoms" in keywords)):
            if "all" in keywords:
                return self.getValues("symptoms appear")
            # It returns the minimum of the bad temperature
            elif len([kw for kw in keywords if kw in self.getValues("how spreads")]) > 0:
                return "Yes"
            elif len(self.getValues("symptoms appear")) > 0:
                return self.getValues("symptoms appear")[0]

        # Parse symptoms 
        elif ("symptom" in keywords or "symptoms" in keywords):
            if "common" in keywords:
                if "all" in keywords or "symptoms" in keywords:
                    return self.getValues("common symptoms")
                # It returns the minimum of the bad temperature
                elif len([kw for kw in keywords if kw in self.getValues("common symptoms")]) > 0:
                    return "Yes"
                elif len(self.getValues("common symptoms")) > 0:
                    return self.getValues("common symptoms")[0]
            elif "serious" in keywords:
                if "all" in keywords or "symptoms" in keywords:
                    return self.getValues("serious symptoms")
                # It returns the minimum of the bad temperature
                elif len([kw for kw in keywords if kw in self.getValues("serious symptoms")]) > 0:
                    return "Yes"
                elif len(self.getValues("serious symptoms")) > 0:
                    return self.getValues("serious symptoms")[0]
            else: # rare symptoms
                if "all" in keywords or "symptoms" in keywords:
                    return self.getValues("rare symptoms")
                # It returns the minimum of the bad temperature
                elif len([kw for kw in keywords if kw in self.getValues("rare symptoms")]) > 0:
                    return "Yes"
                elif len(self.getValues("rare symptoms")) > 0:
                    return self.getValues("rare symptoms")[0]
        # no match
        return None

    ### Other functions ###

    # how the table is printed
    def __repr__(self):
        result = "Information table\n"
        i = 1
        for row in list(self.table.keys()):
            result += str(i) + ". Dimension: "
            result += row
            result += "\n-Values: "
            value_list, description = self.table[row]
            result += str(value_list)
            result += "\n-Description: " + str(description) + "\n\n"
            i += 1
        return result


## Bash

In [16]:
import sys

def loadTable(data):
    data.addValue("Bad temperature", "38")
    data.add("Bad temperature", "38.2", "Fever temperature when having Coronavirus")
    data.add("Bad temperature", "38.5", None)

    data.addValue("Good places", "Parks")
    data.add("Good places", "Open places", "Good places to go when a pandemic")
    data.addValue("Good places", "Countryside")

    data.add("covid description", "Coronavirus disease (COVID-19) is an infectious disease caused by a newly discovered coronavirus", "Definition of Coronavirus")
    data.add("who infects by covid", "Human people infects with the COVID-19", "Target of COVID-19")

    data.add("risk people", "older people", "People with special risks if infected by COVID-19")
    data.add("risk people", "cardiovascular disease")
    data.add("risk people", "diabetes")
    data.add("risk people", "chronic respiratory disease")
    data.add("risk people", "cancer")

    data.add("prevention advice", "be well informed about the virus, the disease it causes, and how it spreads", "Prevention tips for COVID-19")
    data.add("prevention advice", "protect yourself and others from infection by washing your hands or using an alcohol based rub frequently")
    data.add("prevention advice", "not touching your face")
    data.add("prevention advice", "coughing into a flexed elbow")
    data.add("prevention advice", "maintain at least 1 metre distance between you and people coughing or sneezing")
    data.add("prevention advice", "Stay home if you feel unwell")
    
    data.add("Bad places", "You should avoid big cities, and public events.")
    
    data.add("how spreads", "through droplets of saliva", "How the COVID-19 spreads and produces contagions")
    data.add("how spreads", "discharge from the nose when an infected person coughs or sneezes")

    data.add("common symptoms", "Most people infected with the COVID-19 virus will experience mild to moderate respiratory illness", "Common symptoms of COVID-19")

    data.add("common symptoms", "dry cough", "Common COVID-19 symptoms")
    data.add("common symptoms", "fever")
    data.add("common symptoms", "tiredness")

    data.add("rare symptoms", "aches and pains", "Uncommon COVID-19 symptoms")
    data.add("rare symptoms", "sore throat")
    data.add("rare symptoms", "diarrhoea")
    data.add("rare symptoms", "conjunctivitis")
    data.add("rare symptoms", "headache")
    data.add("rare symptoms", "loss of taste or smell")

    data.add("serious symptoms", "difficulty breathing or shortness of breath", "Serious COVID-19 symptoms")
    data.add("serious symptoms", "chest pain or pressure")

    data.add("symptoms appear", "On average it takes 5–6 days from when someone is infected with the virus for symptoms to show, however it can take up to 14 days", "When COVID-19 symptoms appear")

    data.add("Ventilation tips", "large quantities of fresh and clean outdoor air", "Pieces of advise to ventilate places")


class Bash:
    def __init__(self, infersent, corpus):
        # Command lists
        self.PROCESS_QUERY_CMDS = ['1']
        self.ADD_QUESTION_CMDS = ['2']
        self.QUIT_CMDS = ['quit', 'q', 'exit', '6']
        self.CLEAR_CMDS = ['clear', 'clean']
        self.RESET_CMDS = ['reset']
        self.HELP_CMDS = ['-h', 'h', 'help', '5']
        
        self.Corpus = infersent
        self.Questions = corpus
        self.Table = Table()
        loadTable(self.Table)
        
        # Flow control variable
        self.go_on = True
        self.main_loop()

    def main_loop(self):
        # Main console loop
        while self.go_on:
            print('Please enter your choice :\n\n')
            print('    1. Ask a question\n')
            print('    2. Add a question and its answer\n')
            print('    3. Informations\n')
            print('    4. Future updates\n')
            print('    5. Help\n')
            print('    6. Quit\n')
            command = input('> ').strip().lower()
            self.exec_cmd(command)

    # Command functions
    def help_cmd(self):
        print('Sars-engine is a collaborative framework for public-health SARS-CoV-2prevention.')
        print('It is an interactive query-answer system. Select the number of the option to perform an action.')
        print('Write \"clear\" to clean the screen. \"Quit\" to exit the application.\n')

    def clear_cmd(self):
        print('\n'*128)

    def reset_cmd(self):
        global go_on
        print('Reset something')
        self.clear_cmd()
        self.go_on = True

    def process_query_cmd(self, query):
        print('-- Processing query:', query, '--')
        question = input('\nPlease, enter your question.\n').strip()
        answer = self.Table.processQuery(question.lower())
        found = True
        if answer is None or answer is "":
            found = False
        if found: # Manual search success
            if isinstance(answer, list):
                print(', '.join(answer))
            else:
                print(answer)
            self.Questions.process_query(question,0)
        else: # Corpus

            self.Questions.process_query(question,1)

    def add_question_cmd(self):
        self.Questions.add_question_answer()
        
    def exec_cmd(self, command):
        if command in self.QUIT_CMDS:
            self.go_on = False
        elif command in self.HELP_CMDS:
            self.help_cmd()
        elif command in self.CLEAR_CMDS:
            self.clear_cmd()
        elif command in self.RESET_CMDS:
            self.reset_cmd()
        elif command in self.PROCESS_QUERY_CMDS:
            self.process_query_cmd(command)
        elif command in self.ADD_QUESTION_CMDS:
            self.add_question_cmd()
#        elif command in ['3']:
#            TODO informations
#        elif command in ['4']:
#            TODO Future updates
        else:
            print('Invalid Command. Write \'-h\' for help.')
        
        

## Execution

In [21]:
print('-----Welcome to Sars-engine-----\nLoading the models...\n\n')
infersent = InferSent_corpus(path)
question_corpus = Corpus_questions(infersent)

-----Welcome to Sars-engine-----
Loading the models...




In [22]:
mybash = Bash(infersent,question_corpus)

Please enter your choice :


    1. Ask a question

    2. Add a question and its answer

    3. Informations

    4. Future updates

    5. Help

    6. Quit



>  1


-- Processing query: 1 --



Please, enter your question.
 Bad places to go


you should avoid big cities, and public events.


...


Please, enter your question.
  Is there a covid vaccine that prevent from infection ?

This question is not recognized by the database.
 Did you mean : Are covid vaccines 100% effectives to prevent covid ?(yes/no)
 no
You could find the desired by exploring our search engine. Would you like to do it ? (yes/no)
 yes



 Results of page 1 : 

Article 447
Is this the future of vaccine production?

Article 484
Unfortunately there is no vaccine to prevent from novel coronavirus infection.

Article 438
To minimize the circulation of vaccine viruses in, e.g.

Article 203
Vaccination with inactivated influenza vaccine represents the main strategy to prevent infection.

Article 493
There is potential for further investigation of the Ad5-nCoV vaccine for the control of the COVID-19 outbreak.



To get more info on an article, enter its number. To see more results, enter 'More'. To make another search, enter 'Menu'. To go to previous page, enter 'Previous'.

 Menu


Please enter your choice :


    1. Ask a question

    2. Add a question and its answer

    3. Informations

    4. Future updates

    5. Help

    6. Quit



>  6


In [8]:
#define YES and NO tables (if value in YES,...)
#make articles reading readable
#forget pickle and use only joblib
#add weights for questions (if they are frecuently asked, they have more chance to be picked)
#add a research over article titles
#analyze influence of tokenize argument of infersent encoding
#find a way to delete model_path etc from first cell 
#find the way to add the vocabulary of the mqnual corpus's questions in the infersent vocab (not that much important)
#improve menu systems (always be able to go back, etc)
#add consulting every question and answers.